In [1]:
import pandas as pd
import numpy as np
import random
import itertools

In [2]:
class Card:
    SUIT_MAP = {
        'D': 'Diamonds',
        'S': 'Spades',
        'C': 'Clubs',
        'H': 'Hearts'
    }

    RANK_MAP = {
        '2': "Two",
        '3': "Three",
        '4': "Four",
        '5': "Five",
        '6': "Six",
        '7': "Seven",
        '8': "Eight",
        '9': "Nine",
        '10': "Ten",
        'J': "Jack",
        'Q': "Queen",
        'K': "King",
        'A': "Ace"
    }

    COLOR_MAP = {
        'R': "Red",
        'B': "Black"
    }

    COLOR_SUIT_MAP = {
        'R':['H', 'D'],
        'B': ['S', 'C']
    }

    SUIT_COLOR_MAP = {
        'H': 'R',
        'D': 'R',
        'S': 'B',
        'C': 'B'
    }

    def __init__(self, suit: str, rank: str):
        self.suit = suit
        self.color = Card.SUIT_COLOR_MAP[suit]
        self.rank = rank

    def getSuit(self) -> str:
        return self.suit

    def getColor(self) -> str:
        return self.color

    def getRank(self) -> str:
        return self.rank

    def printCard(self):
        name = Card.RANK_MAP[self.rank] + " of " + Card.SUIT_MAP[self.suit]
        print(name)

    def getCard(self):
        return (self.rank + Card.SUIT_MAP[self.suit])

    @classmethod
    # TODO fix for 10
    def fromShorthand(cls, shorthand: str):
        suit = shorthand[1]
        rank = shorthand[0]
        return cls(suit,rank)

In [3]:
class Deck:

    SUITS = ['S', 'H', 'D', 'C']
    
    RANKS = {
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '10': 10,
        'J': 11,
        'Q': 12,
        'K': 13,
        'A': 14
    }

    COLORS = ['R','B']

    def __init__(self):
        deck = list()
        for rank in Deck.RANKS:
            for suit in Deck.SUITS:
                deck.append(Card(suit,rank))
        random.shuffle(deck)
        self.deck = deck
        self.size = len(deck)

    def getDeck(self):
        return self.deck

    def drawCards(self, numberOfDraws: int = 1) -> list:
        assert self.size >= numberOfDraws, "Deck doesn't have enough cards!"

        np.random.seed()
        cardsDrawn = []
        for i in range(numberOfDraws):
            if self.size!=1:
                location = np.random.randint((self.size)-1)
            else:
                location = 0
            cardsDrawn.append(self.deck.pop(location))
            self.size = self.size - 1
        return cardsDrawn

    def shuffleDeck(self):
        random.shuffle(self.deck)



In [70]:
class Hand:

    def __init__(self, deck: Deck, numberOfCards: int = 3):
        self.hand = deck.drawCards(numberOfCards)

    def getHand(self):
        return self.hand
        
    

In [74]:
def getHandRankings():    
    rankings = {}
    suits = Deck.SUITS
    colors = Deck.COLORS
    ranks = Deck.RANKS
    ranksInv = {val:key for key,val in ranks.items()}

    rank = 1

    existingHands = set()

    # Three of a kind (trail)
    for item in list(ranks.keys())[-1::-1]:
        cards = [(item+suit) for suit in suits]
        members = list(itertools.combinations(cards,3))
        for member in members:
            hand = frozenset(member)
            rankings[hand] = rank
            existingHands.add(hand)

        rank+=1

    # Straight flush (pure sequence)
    for item in ['A']:
        for suit in suits:
            cards = [(item2+suit) for item2 in ['A','K','Q']]
            hand = frozenset(cards)
            rankings[hand] = rank
            existingHands.add(hand)

        rank+=1
        for suit in suits:
            cards = [(item2+suit) for item2 in ['A','2','3']]
            hand = frozenset(cards)
            rankings[hand] = rank
            existingHands.add(hand)

        rank+=1
    for item in range(13,3,-1):
        for suit in suits:
            cards = [ranksInv[item]+suit, ranksInv[item-1]+suit, ranksInv[item-2]+suit]
            hand = frozenset(cards)
            rankings[hand] = rank
            existingHands.add(hand)
        rank+=1


    # Straight (sequence)
    for item in ['A']:
        one = ['A'+suit for suit in suits]
        two = ['K'+suit for suit in suits]
        three = ['Q'+suit for suit in suits]
        full = [one,two,three]
        cards = set(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)

        for hand in cards:
    
            rankings[frozenset(hand)] = rank
            existingHands.add(frozenset(hand))

        rank+=1


        one = ['A'+suit for suit in suits]
        two = ['2'+suit for suit in suits]
        three = ['3'+suit for suit in suits]
        full = [one,two,three]
        cards = frozenset(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)

        for hand in cards:
            rankings[frozenset(hand)] = rank
            existingHands.add(frozenset(hand))
        rank+=1

    for item in range(13,3,-1):
        one = [ranksInv[item]+suit for suit in suits]
        two = [ranksInv[item-1]+suit for suit in suits]
        three = [ranksInv[item-2]+suit for suit in suits]
        full = [one, two, three]
        cards = frozenset(itertools.product(*full))
        cards = {frozenset(item2) for item2 in cards}
        cards = cards.difference(existingHands)
        
        for hand in cards:
            rankings[frozenset(hand)] = rank
            existingHands.add(frozenset(hand))
        rank+=1

    # Flush (color)
    for top in range(14,4, -1):
        for mid in range(top-1, 2, -1):
            if top-mid==1:
                for low in range(mid-2, 1, -1):
                    for suit in suits:
                        cards=[ranksInv[top]+suit, ranksInv[mid]+suit, ranksInv[low]+suit]
                        hand = frozenset(cards)
                        rankings[hand] = rank
                        rank+=1
                        existingHands.add(hand)
            else:
                for low in range(mid-1, 1, -1):
                    if ((top==14 and mid==3) and (low==2)):
                        continue
                    else:
                        for suit in suits:
                            cards=[ranksInv[top]+suit, ranksInv[mid]+suit, ranksInv[low]+suit]
                            hand = frozenset(cards)
                            rankings[hand] = rank
                            rank+=1
                            existingHands.add(hand)

    # Pair
    for pair in range(14,1, -1):
        for top in range(14,1, -1):
            if pair==top:
                continue
            else:
                cards = [(ranksInv[pair]+suit) for suit in suits]
                oneTwo = list(itertools.combinations(cards,2))
                three = [ranksInv[top]+suit for suit in suits]
                full = [oneTwo, three]
                cards = list(itertools.product(*full))
                cards = frozenset([(item[0][0],item[0][1],item[1]) for item in cards])
                for hand in cards:
                    hand = frozenset(hand)
                    rankings[hand] = rank
                    rank+=1
                    existingHands.add(hand)

    # No pair (high card)
    for top in range(14,4, -1):
        for mid in range(top-1, 2, -1):
            if top-mid==1:
                for low in range(mid-2, 1, -1):
                    one = [ranksInv[top]+suit for suit in suits]
                    two = [ranksInv[mid]+suit for suit in suits]
                    three = [ranksInv[low]+suit for suit in suits]
                    full = [one,two,three]
                    cards = frozenset(itertools.product(*full))
                    cards = {frozenset(item2) for item2 in cards}
                    cards = cards.difference(existingHands)
                    for hand in cards:
                        hand = frozenset(hand)
                        rankings[hand] = rank
                        existingHands.add(hand)
                    rank+=1
                        
            else:
                for low in range(mid-1, 1, -1):
                    if ((top==14 and mid==3) and (low==2)):
                        continue
                    else:
                        one = [ranksInv[top]+suit for suit in suits]
                        two = [ranksInv[mid]+suit for suit in suits]
                        three = [ranksInv[low]+suit for suit in suits]
                        full = [one,two,three]
                        cards = frozenset(itertools.product(*full))
                        cards = {frozenset(item2) for item2 in cards}
                        cards = cards.difference(existingHands)
                        for hand in cards:
                            hand = frozenset(hand)
                            rankings[hand] = rank
                            existingHands.add(hand)
                        rank+=1

    return rankings


In [75]:
i=1
for key,value in getHandRankings().items():
    print(i, key,value)
    i+=1

1 frozenset({'AH', 'AS', 'AD'}) 1
2 frozenset({'AH', 'AC', 'AS'}) 1
3 frozenset({'AC', 'AS', 'AD'}) 1
4 frozenset({'AH', 'AC', 'AD'}) 1
5 frozenset({'KS', 'KD', 'KH'}) 2
6 frozenset({'KS', 'KH', 'KC'}) 2
7 frozenset({'KS', 'KD', 'KC'}) 2
8 frozenset({'KD', 'KH', 'KC'}) 2
9 frozenset({'QH', 'QD', 'QS'}) 3
10 frozenset({'QH', 'QC', 'QS'}) 3
11 frozenset({'QD', 'QC', 'QS'}) 3
12 frozenset({'QD', 'QC', 'QH'}) 3
13 frozenset({'JS', 'JD', 'JH'}) 4
14 frozenset({'JS', 'JC', 'JH'}) 4
15 frozenset({'JS', 'JD', 'JC'}) 4
16 frozenset({'JD', 'JC', 'JH'}) 4
17 frozenset({'10H', '10D', '10S'}) 5
18 frozenset({'10C', '10H', '10S'}) 5
19 frozenset({'10C', '10D', '10S'}) 5
20 frozenset({'10C', '10D', '10H'}) 5
21 frozenset({'9H', '9S', '9D'}) 6
22 frozenset({'9C', '9H', '9S'}) 6
23 frozenset({'9C', '9D', '9S'}) 6
24 frozenset({'9C', '9H', '9D'}) 6
25 frozenset({'8D', '8H', '8S'}) 7
26 frozenset({'8C', '8H', '8S'}) 7
27 frozenset({'8C', '8D', '8S'}) 7
28 frozenset({'8C', '8H', '8D'}) 7
29 frozenset({'7S

In [61]:
# TODO demonstration

deck = Deck()

print("Drawing 20 cards")
drawn = []
left = []
for item in deck.drawCards(20):
    drawn.append(item.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards drawn:", drawn)
print("Deck remaining:", left)
print()

print("Drawing a hand of 3 cards")
hand = Hand(deck)
handCards = []
left = []
for card in hand.getHand():
    handCards.append(card.getCard())
for item in deck.getDeck():
    left.append(item.getCard())
print("Deck size remaining", deck.size)
print("Cards in hand:", handCards)
print("Deck remaining:", left)
print()

Drawing 20 cards
Deck size remaining 32
Cards drawn: ['J of Diamonds', '9 of Spades', 'K of Spades', '5 of Clubs', '3 of Diamonds', 'A of Hearts', 'K of Clubs', '7 of Spades', 'A of Diamonds', 'Q of Diamonds', 'J of Spades', '2 of Hearts', '6 of Hearts', '9 of Diamonds', 'J of Clubs', '3 of Spades', '7 of Hearts', '6 of Diamonds', '7 of Diamonds', '4 of Spades']
Deck remaining: ['K of Hearts', '9 of Clubs', 'K of Diamonds', '3 of Clubs', '10 of Spades', '2 of Clubs', 'A of Clubs', 'Q of Spades', '4 of Hearts', '5 of Hearts', '8 of Diamonds', '5 of Spades', '4 of Clubs', '10 of Clubs', '7 of Clubs', '3 of Hearts', '6 of Clubs', '9 of Hearts', '8 of Spades', '5 of Diamonds', 'J of Hearts', '4 of Diamonds', 'Q of Hearts', '8 of Hearts', '10 of Hearts', '6 of Spades', '2 of Spades', '10 of Diamonds', '2 of Diamonds', '8 of Clubs', 'Q of Clubs', 'A of Spades']

Drawing a hand of 3 cards
Deck size remaining 29
Cards in hand: ['6 of Clubs', '8 of Hearts', '8 of Clubs']
Deck remaining: ['K of 

In [6]:
class Game:

    HAND_RANKINGS = {}

    def __init__(self, noOfPlayers: int, jokers: list = None):
        self.noOfPlayers = noOfPlayers
        self.hands = self.generateHands()
        self.jokers = jokers

    def generateHands(self):
        pass